In [1]:
import pandas as pd
from tqdm import tqdm
import random
import os

In [2]:
tqdm.pandas()

In [3]:
REPOS = ['Apache', 'Hyperledger', 'IntelDAOS', 'JFrog', 'Jira', 
         'JiraEcosystem', 'MariaDB', 'Mojang', 'MongoDB', 'Qt', 
         'RedHat', 'Sakai', 'SecondLife', 'Sonatype', 'Spring']

In [4]:
CLEANED_ISSUE_DIR = '../data/processed/cleaned_issues/'
LINK_DIR = '../data/raw/links/'

issue_paths = []
for root, dirs, file_names in os.walk(CLEANED_ISSUE_DIR):
    for file_name in file_names:
        issue_paths.append(os.path.join(root, file_name))

In [5]:
def load_issues(repo: str):
    issue_path_ls = [issue_path for issue_path in issue_paths if repo in issue_path]
    issue_df = pd.DataFrame()
    for issue_path in issue_path_ls:
        tmp_df = pd.read_csv(issue_path, sep=';', encoding='utf-8', low_memory=False, 
                             usecols=lambda x : x not in ['created', 'updated', 'comments', 'components'], index_col=['issue_id'])
        issue_df = pd.concat([issue_df, tmp_df])
    return issue_df

In [6]:
def load_links(repo: str):
    link_path = LINK_DIR + repo + '.csv'
    link_df = pd.read_csv(link_path, sep=';', encoding="UTF-8", low_memory=False).drop_duplicates()
    return link_df

In [7]:
# This function is to identify doubled issue pairs
def add_linked_issues_to_df(df):
    df['issues'] = ''
    for i in tqdm(range(len(df))):
        df['issues'].iloc[i] = str(sorted(set([df.iloc[i]['issue_id_1'], df.iloc[i]['issue_id_2']])))

In [8]:
def clean_links(link_df, issue_df):    

    # Remove links with uncrawled and filtered issues
    link_df = link_df[link_df[['issue_id_1', 'issue_id_2']].isin(issue_df.index.values).all(axis=1)]
    print(f'Left with {len(link_df)} links after removing half-private links')
    
    # Cleanup links, only allow one link per issue-pair
    link_df.drop_duplicates(subset=['name'], keep=False, inplace=True)
    print(f'Left with {len(link_df)} links after removing issue-pairs with multiple links between them')

    # In case the name is the otherway around, like issue-1_issue-2 and issue-2_issue-1
    doublelinks = (link_df.issues.value_counts() > 1).rename_axis('doubles').reset_index(name='valid')
    valid_doubles = set(doublelinks[doublelinks['valid'] == True]['doubles'])

    for i in tqdm(valid_doubles):
        if len(set(link_df[link_df['issues'] == i]['linktype'])) > 1:
            link_df = link_df[link_df.issues != i]
    print(f'Left with {len(link_df)} links after removing issue-pairs with multiple link types between them')

    # Multiple links complete remove
    link_df.drop_duplicates(subset=['issues'], inplace=True)
    print(f'Left with {len(link_df)} links after removing issue-pairs with multiple entries')

    link_df.reset_index(inplace=True, drop=True)
    
    return link_df

In [9]:
# Create non-links from randomly selected issues
def create_non_links(link_df, issue_df, linked_pairs):
   
    non_link_issues = set(issue_df[~issue_df['resolution'].isin(['Duplicate'])].index)
    
    cols = ['name', 'linktype', 'issue_id_1', 'issue_id_2', 'issues']
    non_link_df = pd.DataFrame(columns=cols)

    for i in tqdm(range(int(link_df.linktype.value_counts().mean()))):
        sample = random.sample(non_link_issues, 2)
        if not set([sample[0], sample[1]]) in (linked_pairs):
            name = str(sample[0]) + "_" + str(sample[1])
            non_link = pd.DataFrame([{
                "name": name,
                "linktype": "Non-Link",
                "issue_id_1":  sample[0],
                "issue_id_2":  sample[1],
                "issues": str(sorted(set([sample[0], sample[1]])))
            }])
            non_link_df = pd.concat([non_link_df, non_link], ignore_index=True)
        else:
            print('hi')

    link_plus_df = pd.concat([link_df, non_link_df], ignore_index=True)
    
    return link_plus_df

In [10]:
# Join links and issues
def joined_links(link_df, issue_df):
    link_df = link_df[['linktype', 'issue_id_1', 'issue_id_2']]
    joined_df = link_df \
        .join(issue_df.add_suffix('_1'), on='issue_id_1') \
        .join(issue_df.add_suffix('_2'), on='issue_id_2')
    return joined_df

In [11]:
for repo in REPOS:
    issue_df = load_issues(repo)
    link_df = load_links(repo)
    print(f'Loaded {len(issue_df)} issues and {len(link_df)} links for {repo}')
    
    add_linked_issues_to_df(link_df)
    link_df = clean_links(link_df, issue_df)

    print(f'Cleaned {len(link_df)} links for {repo}')
    print(link_df.linktype.value_counts())

    linked_pairs = set(link_df['issues'])
    link_plus_df = create_non_links(link_df, issue_df, linked_pairs)
    
    print(f'Total cleaned links plus non_links is {len(link_plus_df)} for {repo}')

    link_df = joined_links(link_df, issue_df)
    link_df.to_csv('./data/joined/links/' + repo + '.csv', sep=';', encoding='utf-8', index=True)
    
    link_plus_df = joined_links(link_plus_df, issue_df)
    link_plus_df.to_csv('./data/joined/links_plus/' + repo + '.csv', sep=';', encoding='utf-8', index=True)

    print("----------------------------\n")

Loaded 1014926 issues and 264107 links for Apache


100%|██████████| 264107/264107 [00:47<00:00, 5551.72it/s]
/tmp/ipykernel_1717/4275618611.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  link_df.drop_duplicates(subset=['name'], keep=False, inplace=True)


Left with 263647 links after removing half-private links
Left with 259717 links after removing issue-pairs with multiple links between them


100%|██████████| 2218/2218 [00:52<00:00, 41.85it/s]


Left with 256253 links after removing issue-pairs with multiple link types between them
Left with 255767 links after removing issue-pairs with multiple entries
Cleaned 255767 links for Apache
Subtask             83783
Reference           68973
Duplicate           25925
Blocker             14377
Epic-Relation       12506
dependent           12498
Incorporates         6923
Regression           4350
Cloners              4321
Required             3620
Container            3508
Related              3369
Supercedes           3248
Problem/Incident     3106
Child-Issue          2508
Blocked              1172
Completes             914
Dependent             399
Dependency            112
Testing                79
Parent Feature         50
Issue split            26
Name: linktype, dtype: int64


...

Left with 14616 links after removing half-private links
Left with 14550 links after removing issue-pairs with multiple links between them


100%|██████████| 52/52 [00:00<00:00, 722.61it/s]


Left with 14478 links after removing issue-pairs with multiple link types between them
Left with 14462 links after removing issue-pairs with multiple entries
Cleaned 14462 links for Spring
Relate           5909
Subtask          1941
Duplicate        1745
Epic-Relation    1635
Depend           1259
Related           993
Depends           488
Supersede         478
Cloners            14
Name: linktype, dtype: int64


100%|██████████| 1606/1606 [00:02<00:00, 767.14it/s]


Total cleaned links plus non_links is 16068 for Spring
----------------------------

